http://www.yeastgenome.org/locus/S000000747/overview
## Number of Pyramidines in URA3 gene:  148
## Number of Pyramidines when minimize: 121
## Number of Pyramidines when maximize: 195

# version as of Jan 20th:  
## also max/min purines, to account for the second strand
## when possible, keep the codons the same as in the original version
hi

In [74]:
import csv
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sys

aa_to_syn_codons_dict = dict()
codon_to_aa_dict = dict()
csvfile = open('AA_CODONS_MAP')
reader = csv.reader(csvfile, delimiter='\t')
for row in reader:
    mylist = row[3].split(',')
    aa_to_syn_codons_dict[row[0]] = mylist
    for codon in mylist:
        codon_to_aa_dict[codon] = row[0]

def num_pyramidine_dimers(seq):
    num = 0
    for i in range(0,(len(seq)-1)):
        if ((seq[i] == "T" or seq[i] == "C") and (seq[i+1] == "T" or seq[i+1] == "C")):
            num += 1
    return(num)

def num_pyramidine_or_purine_dimers(seq):
    num = 0
    for i in range(0,(len(seq)-1)):
        if ((seq[i] == "T" or seq[i] == "C") and (seq[i+1] == "T" or seq[i+1] == "C")):
            num += 1
        elif ((seq[i] == "A" or seq[i] == "G") and (seq[i+1] == "A" or seq[i+1] == "G")):
            num += 1
    return(num)

URA_3_codons = "ATGTCGAAAGCTACATATAAGGAACGTGCTGCTACTCATCCTAGTCCTGTTGCTGCCAAG"\
                "CTATTTAATATCATGCACGAAAAGCAAACAAACTTGTGTGCTTCATTGGATGTTCGTACC"\
                "ACCAAGGAATTACTGGAGTTAGTTGAAGCATTAGGTCCCAAAATTTGTTTACTAAAAACA"\
                "CATGTGGATATCTTGACTGATTTTTCCATGGAGGGCACAGTTAAGCCGCTAAAGGCATTA"\
                "TCCGCCAAGTACAATTTTTTACTCTTCGAAGACAGAAAATTTGCTGACATTGGTAATACA"\
                "GTCAAATTGCAGTACTCTGCGGGTGTATACAGAATAGCAGAATGGGCAGACATTACGAAT"\
                "GCACACGGTGTGGTGGGCCCAGGTATTGTTAGCGGTTTGAAGCAGGCGGCGGAAGAAGTA"\
                "ACAAAGGAACCTAGAGGCCTTTTGATGTTAGCAGAATTGTCATGCAAGGGCTCCCTAGCT"\
                "ACTGGAGAATATACTAAGGGTACTGTTGACATTGCGAAGAGCGACAAAGATTTTGTTATC"\
                "GGCTTTATTGCTCAAAGAGACATGGGTGGAAGAGATGAAGGTTACGATTGGTTGATTATG"\
                "ACACCCGGTGTGGGTTTAGATGACAAGGGAGACGCATTGGGTCAACAGTATAGAACCGTG"\
                "GATGATGTGGTCTCTACAGGATCTGACATTATTATTGTTGGAAGAGGACTATTTGCAAAG"\
                "GGAAGGGATGCTAAGGTAGAGGGTGAACGTTACAGAAAAGCAGGCTGGGAAGCATATTTG"\
                "AGAAGATGCGGCCAGCAAAACTAA"
#URA_3_codons = "ATGTCTACG"
URA_3_AA = "MSKATYKERAATHPSPVAAKLFNIMHEKQTNLCASLDVRTTKELLELVEALGPKICLLKT"\
                    "HVDILTDFSMEGTVKPLKALSAKYNFLLFEDRKFADIGNTVKLQYSAGVYRIAEWADITN"\
                    "AHGVVGPGIVSGLKQAAEEVTKEPRGLLMLAELSCKGSLATGEYTKGTVDIAKSDKDFVI"\
                    "GFIAQRDMGGRDEGYDWLIMTPGVGLDDKGDALGQQYRTVDDVVSTGSDIIIVGRGLFAK"\
                    "GRDAKVEGERYRKAGWEAYLRRCGQQN*"
#URA_3_AA = "MST"
store_best = [None] * len(URA_3_AA)
for i in range(0,len(URA_3_AA)):
    store_best[i] = dict()
    for codon in aa_to_syn_codons_dict[URA_3_AA[i]]:
        store_best[i][codon] = 0
                
# do the last AA
for codon in store_best[-1]:
    store_best[i][codon] = (num_pyramidine_or_purine_dimers(codon),"")


# do the dynamic program, stating from the one before the end of the list, until the end
for pos in reversed(range(0, len(store_best)-1)):
    for codon in store_best[pos]:
        # consider as cost the cost of adding this codon to one of the best sequences
        # from the previous suffix
        # maximizer the dimers
        best_prev = ""
        best_cost = -1 #sys.maxsize #-1
        for prev_codon in store_best[pos+1]:
            cost = num_pyramidine_or_purine_dimers(codon+prev_codon[0]) + store_best[pos+1][prev_codon][0]
            # maximizer the dimers
            if ((cost > best_cost) or # > <
                ((cost == best_cost) and (prev_codon == URA_3_codons[(pos+1)*3:(pos+1)*3+3]))): #try to keep same seq
                best_cost = cost
                best_prev = prev_codon
        store_best[pos][codon] = (best_cost, best_prev)
        #print("storing for " + str(pos) + " " + codon + ": " + str(best_cost) + " " + best_prev)

recoded_sequence = ""
next_codon = ""
for pos in range(0, len(store_best)):
    if pos == 0:
        # maximum
        best_first = ""
        best_first_cost = -1 #sys.maxsize #-1
        for codon in store_best[0]:
            # maximum
            if ((store_best[pos][codon][0] > best_first_cost) or  #> <
                ((cost == best_cost) and (codon == URA_3_codons[(pos+1)*3:(pos+1)*3+3]))): #try to keep same seq
                best_first = codon
                best_first_cost = store_best[pos][codon][0]
                next_codon = store_best[pos][codon][1]
        recoded_sequence += best_first
    else:
        recoded_sequence += next_codon
        next_codon = store_best[pos][next_codon][1]

#print(URA_3_codons)
print(recoded_sequence)    
print("num pyramidine dimers: " + str(num_pyramidine_dimers(recoded_sequence)))
back_to_original = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (recoded_sequence[i:i+3] != URA_3_codons[i:i+3]):
        diff_total += 1
    back_to_original += codon_to_aa_dict[recoded_sequence[i:i+3]]
print("total number of codon diferences between original and recoded sequences " + str(diff_total))
if (URA_3_AA == back_to_original):
    print("AA sequence remains the same, good")
#print(URA_3_codons)
#print("num pyramidine dimers in original sequences: " + str(num_pyramidine_dimers(URA_3_codons)))
print("num purine and pyramidine dimers in original sequences: " + str(num_pyramidine_or_purine_dimers(URA_3_codons)))
print("num purine and pyramidine dimers in recoded sequences: " + str(num_pyramidine_or_purine_dimers(recoded_sequence)))
print("length recoded sequence: " + str(len(recoded_sequence)))

ATGTCGAAAGCTACTTATAAGGAAAGAGCTGCTACTCATCCTTCTCCTGTTGCTGCCAAGCTCTTTAATATCATGCACGAAAAGCAAACAAACCTCTGTGCTTCTTTGGATGTTCGGACCACCAAGGAACTCCTGGAGTTAGTTGAAGCTTTAGGTCCCAAAATTTGTCTCCTAAAAACTCATGTGGATATCTTGACTGATTTTTCCATGGAGGGGACAGTTAAGCCTCTAAAGGCTCTCTCCGCCAAGTACAATTTTCTCCTCTTCGAAGACAGAAAATTTGCTGACATTGGGAATACAGTCAAACTCCAGTACTCTGCGGGGGTTTACAGAATAGCAGAATGGGCAGACATTACGAATGCTCACGGGGTGGTGGGCCCAGGGATTGTTTCTGGTTTGAAGCAGGCGGCGGAAGAAGTAACAAAGGAACCTAGAGGCCTTTTGATGTTAGCAGAACTCTCTTGCAAGGGCTCCCTAGCTACTGGAGAATATACTAAGGGGACTGTTGACATTGCGAAGAGCGACAAAGATTTTGTTATCGGCTTTATTGCTCAAAGAGACATGGGGGGAAGAGATGAAGGTTACGATTGGTTGATTATGACTCCCGGGGTGGGTTTAGATGACAAGGGAGACGCTTTGGGTCAACAGTATAGAACCGTGGATGATGTGGTCTCTACAGGATCTGACATTATTATTGTTGGAAGAGGACTCTTTGCAAAGGGAAGGGATGCTAAGGTAGAGGGGGAAAGATACAGAAAAGCAGGCTGGGAAGCTTATTTGAGAAGATGCGGCCAGCAAAACTAA
num pyramidine dimers: 195
total number of codon diferences between original and recoded sequences 35
AA sequence remains the same, good
num purine and pyramidine dimers in original sequences: 41

## version as of Jan 17th
## Number of Pyramidine dimers in URA3 gene: 148
## Number of Pyramidine dimers when minimize: 63
## Number of Pyramidine dimers when maximize: 252

In [73]:
import csv
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sys

aa_to_syn_codons_dict = dict()
codon_to_aa_dict = dict()
csvfile = open('AA_CODONS_MAP')
reader = csv.reader(csvfile, delimiter='\t')
for row in reader:
    mylist = row[3].split(',')
    aa_to_syn_codons_dict[row[0]] = mylist
    for codon in mylist:
        codon_to_aa_dict[codon] = row[0]

def num_pyramidine_dimers(seq):
    num = 0
    for i in range(0,(len(seq)-1)):
        if ((seq[i] == "T" or seq[i] == "C") and (seq[i+1] == "T" or seq[i+1] == "C")):
            num += 1
    return(num)

URA_3_codons = "ATGTCGAAAGCTACATATAAGGAACGTGCTGCTACTCATCCTAGTCCTGTTGCTGCCAAG"\
                "CTATTTAATATCATGCACGAAAAGCAAACAAACTTGTGTGCTTCATTGGATGTTCGTACC"\
                "ACCAAGGAATTACTGGAGTTAGTTGAAGCATTAGGTCCCAAAATTTGTTTACTAAAAACA"\
                "CATGTGGATATCTTGACTGATTTTTCCATGGAGGGCACAGTTAAGCCGCTAAAGGCATTA"\
                "TCCGCCAAGTACAATTTTTTACTCTTCGAAGACAGAAAATTTGCTGACATTGGTAATACA"\
                "GTCAAATTGCAGTACTCTGCGGGTGTATACAGAATAGCAGAATGGGCAGACATTACGAAT"\
                "GCACACGGTGTGGTGGGCCCAGGTATTGTTAGCGGTTTGAAGCAGGCGGCGGAAGAAGTA"\
                "ACAAAGGAACCTAGAGGCCTTTTGATGTTAGCAGAATTGTCATGCAAGGGCTCCCTAGCT"\
                "ACTGGAGAATATACTAAGGGTACTGTTGACATTGCGAAGAGCGACAAAGATTTTGTTATC"\
                "GGCTTTATTGCTCAAAGAGACATGGGTGGAAGAGATGAAGGTTACGATTGGTTGATTATG"\
                "ACACCCGGTGTGGGTTTAGATGACAAGGGAGACGCATTGGGTCAACAGTATAGAACCGTG"\
                "GATGATGTGGTCTCTACAGGATCTGACATTATTATTGTTGGAAGAGGACTATTTGCAAAG"\
                "GGAAGGGATGCTAAGGTAGAGGGTGAACGTTACAGAAAAGCAGGCTGGGAAGCATATTTG"\
                "AGAAGATGCGGCCAGCAAAACTAA"
#original_sequence = "DICCIPSSSH"
original_sequence = "MSKATYKERAATHPSPVAAKLFNIMHEKQTNLCASLDVRTTKELLELVEALGPKICLLKT"\
                    "HVDILTDFSMEGTVKPLKALSAKYNFLLFEDRKFADIGNTVKLQYSAGVYRIAEWADITN"\
                    "AHGVVGPGIVSGLKQAAEEVTKEPRGLLMLAELSCKGSLATGEYTKGTVDIAKSDKDFVI"\
                    "GFIAQRDMGGRDEGYDWLIMTPGVGLDDKGDALGQQYRTVDDVVSTGSDIIIVGRGLFAK"\
                    "GRDAKVEGERYRKAGWEAYLRRCGQQN*"
store_best = [None] * len(original_sequence)
for i in range(0,len(original_sequence)):
    store_best[i] = dict()
    for codon in aa_to_syn_codons_dict[original_sequence[i]]:
        store_best[i][codon] = 0
                
# do the last AA
for codon in store_best[-1]:
    store_best[i][codon] = (num_pyramidine_dimers(codon),"")


# do the dynamic program, stating from the one before the end of the list, until the end
for pos in reversed(range(0, len(store_best)-1)):
    for codon in store_best[pos]:
        # consider as cost the cost of adding this codon to one of the best sequences
        # from the previous suffix
        # maximizer the dimers
        best_prev = ""
        best_cost = -1 #sys.maxsize #-1
        for prev_codon in store_best[pos+1]:
            cost = num_pyramidine_dimers(codon+prev_codon[0]) + store_best[pos+1][prev_codon][0]
            # maximizer the dimers
            if (cost > best_cost): #> <
                best_cost = cost
                best_prev = prev_codon
        store_best[pos][codon] = (best_cost, best_prev)

recoded_sequence = ""
next_codon = ""
for pos in range(0, len(store_best)):
    if pos == 0:
        # maximum
        best_first = ""
        best_first_cost = -1 #sys.maxsize #-1
        for codon in store_best[0]:
            # maximum
            if (store_best[pos][codon][0] > best_first_cost): #> <
                best_first = codon
                best_first_cost = store_best[pos][codon][0]
                next_codon = store_best[pos][codon][1]
        #print(best_first_cost)
        recoded_sequence += best_first
    else:
        recoded_sequence += next_codon
        next_codon = store_best[pos][next_codon][1]
print(recoded_sequence)    
print(num_pyramidine_dimers(recoded_sequence))
#back_to_original = ""
#for i in range(0,len(recoded_sequence),3):
#    back_to_original += codon_to_aa_dict[recoded_sequence[i:i+3]]
if (original_sequence == back_to_original):
    print("AA sequence remains the same, good")
print(URA_3_codons)
print(num_pyramidine_dimers(URA_3_codons))

ATGTCCAAGGCCACTTATAAGGAGCGGGCCGCCACTCATCCTTCCCCTGTCGCCGCCAAGCTCTTTAACATCATGCATGAGAAGCAAACTAACCTCTGCGCCTCCCTCGATGTCCGGACTACTAAGGAGCTCCTCGAGCTCGTCGAGGCCCTCGGCCCTAAGATCTGCCTCCTCAAGACTCATGTCGATATCCTCACTGATTTTTCCATGGAGGGAACTGTCAAGCCTCTCAAGGCCCTCTCCGCCAAGTATAACTTTCTCCTCTTTGAGGATCGGAAGTTTGCCGATATCGGAAACACTGTCAAGCTCCAATATTCCGCCGGAGTCTATCGGATCGCCGAGTGGGCCGATATCACTAACGCCCATGGAGTCGTCGGCCCTGGAATCGTCTCCGGCCTCAAGCAAGCCGCCGAGGAGGTCACTAAGGAGCCTCGGGGCCTCCTCATGCTCGCCGAGCTCTCCTGCAAGGGCTCCCTCGCCACTGGAGAGTATACTAAGGGAACTGTCGATATCGCCAAGTCCGATAAGGATTTTGTCATCGGCTTTATCGCCCAACGGGATATGGGAGGCCGGGATGAGGGCTATGATTGGCTCATCATGACTCCTGGAGTCGGCCTCGATGATAAGGGAGATGCCCTCGGCCAACAATATCGGACTGTCGATGATGTCGTCTCCACTGGCTCCGATATCATCATCGTCGGCCGGGGCCTCTTTGCCAAGGGCCGGGATGCCAAGGTCGAGGGAGAGCGCTATCGGAAGGCCGGCTGGGAGGCCTATCTCCGCCGCTGCGGCCAACAAAACTAA
252
AA sequence remains the same, good
ATGTCGAAAGCTACATATAAGGAACGTGCTGCTACTCATCCTAGTCCTGTTGCTGCCAAGCTATTTAATATCATGCACGAAAAGCAAACAAACTTGTGTGCTTCATTGGATGTTCGTACCACCAAGGAATTACTGGAGTTAGTTGAAGCATTAGGT